# Description

In [ ]:
import os
import pandas as pd
from datetime import datetime
import numpy as np
import matplotlib.pyplot as plt
import math
from statsmodels.stats.weightstats import DescrStatsW
from scipy.stats import t
import scipy.optimize as optim
import random

In [ ]:
pd.set_option('max_columns', 100)
pd.set_option('max_rows', 150)

In [ ]:
os.getcwd()

In [ ]:
path = '/Users/alvastrand/Documents/OU/Research/data/'
os.chdir(path)

In [ ]:
os.getcwd()

In [ ]:
start_date = '0101'
end_date = '0731'
start_year = '2003'
end_year = '2019'
month = 'Apr'
year_ebird = '2020'
countries_states = 'US_states_east_Mississippi'

In [ ]:
subdir = 'output/'
filename = 'obligate_aerial_insectivores_ebird_species_codes.csv'

df_species_codes = pd.read_csv(subdir + filename)

print(len(df_species_codes))

In [ ]:
df_species_codes.head(2)

In [ ]:
for i in range(len(df_species_codes)):
      
    print(i, df_species_codes['species_code'].iloc[i], df_species_codes['common_name'].iloc[i])

In [ ]:
def get_first_of_season_arrival_day(species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, 
                                    df, list_grid_cells, list_years, *args):
    
    list_grid_cells_first_of_season = []
    list_years_first_of_season = []
    list_t_mad = []
    list_mad = []

    for i in range(len(list_grid_cells)):

#         print('i =', i)

        grid_cell = list_grid_cells[i]
        year = list_years[i]
#         print('(grid_cell, year) =', (grid_cell, year))
        
        start_month = 1
        start_day = 1

        end_month = 7
        end_day = 31
        
        start_date_dt = datetime(year, start_month, start_day)
        end_date_dt = datetime(year, end_month, end_day)
        date_range = pd.date_range(start_date_dt, end_date_dt)
        date_range_str = date_range.strftime('%Y-%m-%d')
        
        df_date_range = pd.DataFrame({'grid_cell': grid_cell, 'year': year, 'observation_date': date_range_str, 
                                      'observation_date_dt': date_range})

        # Get data for a given grid cell and year
        df_block_year = df[(df['grid_cell'] == grid_cell) & (df['year'] == year)]
        df_block_year = df_block_year.reset_index(drop=True)
#         print('len(df_block_year) =', len(df_block_year))

        df_block_year = df_date_range.merge(df_block_year, how='left', on=['grid_cell', 'year', 'observation_date', 
                                                                           'observation_date_dt'])
        df_block_year = df_block_year.reset_index()
        df_block_year = df_block_year.rename(columns={'index': 't'})
        df_block_year['t'] = df_block_year['t'] + 1
#         print('len(df_block_year) =', len(df_block_year))

        start_index = df_block_year[df_block_year['prop'].notna()].head(1).index[0]
#         print('start_index =', start_index)

        df_block_year = df_block_year.iloc[start_index:]
#         print('len(df_block_year) =', len(df_block_year))

        window_size = 7
        # window_size?
        end_index = start_index + window_size - 1
#         print('end_index =', end_index)

        confidence = 0.95

        means = []
        sems = []
        hs = []
        lower_bounds = []
        upper_bounds = []
        
        moving_index = start_index
#         print('moving_index =', moving_index)

        while moving_index < len(df_block_year) - window_size + 1:
            
            df_sample = df_block_year.iloc[range(moving_index, end_index + 1)]
            df_sample = df_sample.dropna()

            if len(df_sample) > 1:

                weights = df_sample['nb_checklists']
                
                dof = 0
                # ?
                # dof = len(df_sample) - 1?
                # Multiply len(df_sample) or some other quantity by the total number of checklists?

                weighted_stats = DescrStatsW(df_sample['prop_arcsine'], weights=weights, ddof=dof)

                mean = weighted_stats.mean
                
                sem = weighted_stats.std_mean

                h = sem * t.ppf((1 + confidence)/2, len(df_sample) - 1)
                # len(df_sample) - 1?
                lower_bound = mean - h
                upper_bound = mean + h

                means.append(mean)
                sems.append(sem)
                hs.append(h)
                lower_bounds.append(lower_bound)
                upper_bounds.append(upper_bound)

            else:

                means.append(np.nan)
                # ?
                sems.append(np.nan)
                # ?
                hs.append(np.nan)
                lower_bounds.append(np.nan)
                upper_bounds.append(np.nan)

            moving_index += 1
            end_index += 1

        # Index of the first mean that is greater than zero
        first_greater_than_zero_index = next((i for i, mean in enumerate(means) if mean > 0), None)
#         print('first_greater_than_zero_index:', first_greater_than_zero_index)
    
#         If there's at least one mean greater than zero:
        if first_greater_than_zero_index != None:
        
#             means[first_greater_than_zero_index] should be greater than zero 
#             (the first value that is greater than zero).
#             print('means[first_greater_than_zero_index]:', means[first_greater_than_zero_index])

            first_greater_than_zero_upper_bound = upper_bounds[first_greater_than_zero_index]
#             print('first_greater_than_zero_upper_bound:', first_greater_than_zero_upper_bound)

            first_greater_than_upper_bound_index = next((i for i, mean in enumerate(means) if 
                                                         mean > first_greater_than_zero_upper_bound), None)
#             print('first_greater_than_upper_bound_index:', first_greater_than_upper_bound_index)

#             If there's at least one proportion that exceeds the upper bound of the confidence interval:
            if first_greater_than_upper_bound_index != None:
        
#                 print('means[first_greater_than_upper_bound_index]:', means[first_greater_than_upper_bound_index])

                index = first_greater_than_upper_bound_index + window_size - 1
#                 print('index:', index)

                t_mad = df_block_year.iloc[index]['t']
#                 print('t_mad:', t_mad)

                mad = df_block_year.iloc[index]['observation_date']
#                 print('mad:', mad)
                
#                 print(df_block_year.iloc[index])
                
#                 print(df_block_year.iloc[range(first_greater_than_zero_index, 
#                                                first_greater_than_zero_index + window_size)])
                
#                 print(df_block_year.iloc[range(first_greater_than_upper_bound_index, 
#                                                first_greater_than_upper_bound_index + window_size)])
                
            # If there aren't any proportions that exceed the upper bound of the confidence interval:
            elif first_greater_than_upper_bound_index == None:

                t_mad = np.nan
                mad = np.nan
        
#             print(len(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt']))
#             print(len(means))
#             print(len(upper_bounds))
            
            assert(len(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
                'observation_date_dt']) == len(means))
        
#             # Means and upper bounds
#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt'], means)
#             plt.scatter(df_block_year.iloc[range(start_index, len(df_block_year) - window_size + 1)][
#                 'observation_date_dt'], upper_bounds)
#             plt.show()

#             # Proportions
#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], df_block_year['prop'])
#             plt.show()

#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], df_block_year['prop_arcsine'])
#             plt.show()

        # If there aren't any means greater than zero:
        elif first_greater_than_zero_index == None:

            t_mad = np.nan
            mad = np.nan

        list_grid_cells_first_of_season.append(grid_cell)
        list_years_first_of_season.append(year)
        list_t_mad.append(t_mad)
        list_mad.append(mad)

    df_first_of_season = pd.DataFrame({'grid_cell': list_grid_cells_first_of_season, 
                                       'year': list_years_first_of_season, 
                                       'first_of_season_arrival_day': list_t_mad, 
                                       'first_of_season_arrival_date': list_mad})

    print('len(df_first_of_season):', len(df_first_of_season))
    
    # Filter

    df_first_of_season = df_first_of_season.dropna()

    print('len(df_first_of_season):', len(df_first_of_season))

    subdir = 'eBird/ebd_output/'

    if args != ():

        countries_states = args[0]
        
        if sampled == 1:
            string = 'sampled'
        elif sampled == 0:
            string = 'not_sampled'
            
        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_proportions_first_of_season_' + str(start_year) + '_' + str(end_year) + \
        '_' + string + '_rel' + month + '-' + year_ebird + '.csv'
        print(filename)

    df_first_of_season.to_csv(subdir + filename, index=False)

    return df_first_of_season, df_block_year

In [ ]:
def logistic_function(t, a, b, c):
    return c/(1 + a * np.exp(-b * t))

In [ ]:
def get_mean_arrival_day(species, start_date, end_date, month, year_ebird, df, list_grid_cells, list_years, *args):

    bounds = (0, [1000000, 100, 1])

    list_grid_cells_logistic = []
    list_years_logistic = []
    list_p0 = []
    list_popt = []
    list_r_squared = []
    list_t_mad = []
    list_mad = []
    list_lower_bound_c = []
    list_upper_bound_c = []
    list_lower_bound_t = []
    list_upper_bound_t = []
    list_ci_nb_days = []
    cnt_errors = 0

    for i in range(len(list_grid_cells)):

        print(i)

        grid_cell = list_grid_cells[i]
        year = list_years[i]
        print(grid_cell, year)

        start_date_dt = datetime(year, start_month, start_day)
        end_date_dt = datetime(year, end_month, end_day)

        date_range = pd.date_range(start_date_dt, end_date_dt)
        date_range_str = date_range.strftime('%Y-%m-%d')

        # Get data for a given grid cell and year
        df_block_year = df[(df['grid_cell'] == grid_cell) & (df['year'] == year)]
        df_block_year.reset_index(drop=True, inplace=True)

        df_date_range = pd.DataFrame({'grid_cell': grid_cell, 'year': year, 'observation_date': date_range_str, 
                                      'observation_date_dt': date_range})

        df_block_year = df_date_range.merge(df_block_year, how='left', on=['grid_cell', 'year', 'observation_date', 
                                                                           'observation_date_dt'])
        df_block_year = df_block_year.reset_index()
        df_block_year.rename(columns={'index': 't'}, inplace=True)
        df_block_year['t'] = df_block_year['t'] + 1
        df_block_year_nas = df_block_year.copy()
        df_block_year = df_block_year.dropna()

        p0 = []
        for j in range(3):
            p0.append(random.uniform(0, 1))
        print(p0)
        # ?

        x = df_block_year['t']
        y = df_block_year['prop']

        try:
            # Nonlinear least squares optimization
            popt, pcov = optim.curve_fit(logistic_function, x, y, bounds=bounds, p0=p0)

            a_optim, b_optim, c_optim = popt
            print(a_optim, b_optim, c_optim)

#             plt.figure(figsize=(12.0, 4.0))
#             plt.scatter(df_block_year['observation_date_dt'], y)
#             plt.plot(df_block_year['observation_date_dt'], logistic_function(x, a_optim, b_optim, c_optim))
#             plt.show()

            residuals = y - logistic_function(x, a_optim, b_optim, c_optim)
            ss_res = np.sum(residuals**2)
            ss_tot = np.sum((y - np.mean(y))**2)
            r_squared = 1 - (ss_res / ss_tot)
            print(r_squared)

            # Time step for mean arrival date
            t_mad = np.log(a_optim) / b_optim
            print(t_mad)

            # Mean arrival date
            mad_floor = df_block_year_nas[df_block_year_nas['t'] == round(t_mad)]['observation_date'].values[0]
    #         df_block_year_nas?
            print(mad_floor)

    #         # Mean arrival date, rounded down
    #         mad_floor = df_block_year_nas[df_block_year_nas['t'] == math.floor(t_mad)]['observation_date'].values[0]
    #         print(mad_floor)

    #         # Mean arrival date, rounded up
    #         mad_ceil = df_block_year_nas[df_block_year_nas['t'] == math.ceil(t_mad)]['observation_date'].values[0]
    #         print(mad_ceil)

            lower_bound_c = (2.5 / 100) * c_optim
            print(lower_bound_c)

            upper_bound_c = (97.5 / 100) * c_optim
            print(upper_bound_c)

            lower_bound_t = -np.log((1 / a_optim)*(c_optim / lower_bound_c - 1)) / b_optim
            print(lower_bound_t)

            upper_bound_t = -np.log((1 / a_optim)*(c_optim / upper_bound_c - 1)) / b_optim
            print(upper_bound_t)

            print(upper_bound_t - lower_bound_t)

    #         lower_bound_date = df_block_year[df_block_year['prop'] >= lower_bound_c]['observation_date_dt'].min()
    #         print(lower_bound_date)

    #         upper_bound_date = df_block_year[df_block_year['prop'] <= upper_bound_c]['observation_date_dt'].max()
    #         print(upper_bound_date)

            list_grid_cells_logistic.append(grid_cell)
            list_years_logistic.append(year)
            list_p0.append(p0)
            list_r_squared.append(r_squared)
            list_t_mad.append(t_mad)
            list_mad.append(mad_floor)
            list_lower_bound_c.append(lower_bound_c)
            list_upper_bound_c.append(upper_bound_c)
            list_lower_bound_t.append(lower_bound_t)
            list_upper_bound_t.append(upper_bound_t)
            list_ci_nb_days.append(upper_bound_t - lower_bound_t)

        except (RuntimeError, IndexError) as e:
            cnt_errors += 1
            assert(
                (e.args[0] == 
                 'Optimal parameters not found: The maximum number of function evaluations is exceeded.') | 
                (e.args[0] == 
                 'index 0 is out of bounds for axis 0 with size 0'))

    print(cnt_errors)

    df_logistic = pd.DataFrame({'grid_cell': list_grid_cells_logistic, 'year': list_years_logistic, 
                                'r_squared': list_r_squared, 't_mad': list_t_mad, 'mad': list_mad, 
                                'ci_nb_days': list_ci_nb_days})

    print(df_logistic.shape)
    
    subdir = 'eBird/ebd_output/'
    
    if args != ():

        countries_states = args[0]
        
        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_proportions_mean_rel' + month + '-' + year_ebird + '_v2.csv'

#         filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
#         '_complete_zerofilled_grid_cells_proportions_mean_rel' + month + '-' + year_ebird + '.csv'
        
        print(filename)

#     df_logistic.to_csv(subdir + filename, index=False)

In [ ]:
def get_arrival_days(species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, *args):
    
    subdir = 'eBird/ebd_output/'

    if args != ():

        countries_states = args[0]
        
        if sampled == 1:
            string = 'sampled'
        elif sampled == 0:
            string = 'not_sampled'

        filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
        '_complete_zerofilled_grid_cells_proportions_' + str(start_year) + '_' + str(end_year) + '_' + string + \
        '_rel' + month + '-' + year_ebird + '.csv'
        print(filename)

    df = pd.read_csv(subdir + filename)
    print('len(df) =', len(df))
    
    df = df.drop(columns=['t'])
    
    df['observation_date_dt'] = pd.to_datetime(df['observation_date'], errors='coerce')
    
    df = df.dropna(subset=['observation_date_dt'])
    print('len(df) =', len(df))

#     df_prop_greater_than = df[df['prop'] > 0]
#     print('len(df_prop_greater_than):', len(df_prop_greater_than))

#     # Calculate the number of proportions greater than 0 for each grid cell and year
#     df_prop_greater_than_cnt = df_prop_greater_than[['grid_cell', 'year', 'prop']].groupby(
#         ['grid_cell', 'year']).count()
#     df_prop_greater_than_cnt = df_prop_greater_than_cnt.reset_index()
#     df_prop_greater_than_cnt = df_prop_greater_than_cnt.rename(columns={'prop': 'nb_prop'})
#     print('len(df_prop_greater_than_cnt):', len(df_prop_greater_than_cnt))

    # Filter
    
#     df_subset = df_prop_greater_than_cnt[df_prop_greater_than_cnt['nb_prop'] >= 10]

    df_grid_cells_years = df[['grid_cell', 'year']].drop_duplicates().sort_values(by=['grid_cell', 'year'])

    list_grid_cells = list(df_grid_cells_years['grid_cell']) # df_subset
    print('len(list_grid_cells) =', len(list_grid_cells))
    print('len(list(set(list_grid_cells))) =', len(list(set(list_grid_cells))))

    list_years = list(df_grid_cells_years['year']) # df_subset

    df_first_of_season, df_block_year = get_first_of_season_arrival_day(
        species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, df, list_grid_cells, 
        list_years, *args)
    
#     get_mean_arrival_day(species, start_date, end_date, month, year_ebird, df, list_grid_cells, list_years, *args)
    
    return df, df_grid_cells_years, df_block_year, df_first_of_season

In [ ]:
species = 'treswa'

sampled = 1

df_sampled, df_grid_cells_years_sampled, df_block_year_sampled, df_first_of_season_sampled = get_arrival_days(
    species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states)

sampled = 0

df_not_sampled, df_grid_cells_years_not_sampled, df_block_year_not_sampled, \
df_first_of_season_not_sampled = get_arrival_days(
    species, start_date, end_date, start_year, end_year, month, year_ebird, sampled, countries_states)

In [ ]:
df_sampled.head(2)

In [ ]:
df_not_sampled.head(2)

In [ ]:
df_grid_cells_years_sampled.head(2)

In [ ]:
df_grid_cells_years_not_sampled.head(2)

In [ ]:
df_block_year_sampled.head(2)

In [ ]:
df_block_year_not_sampled.head(2)

In [ ]:
df_first_of_season_sampled.head(2)

In [ ]:
df_first_of_season_not_sampled.head(2)

In [ ]:
# df['nb_checklists'].value_counts(sort=False).to_frame().reset_index().sort_values(by=['index']).rename(
#     columns={'index': 'nb_checklists', 'nb_checklists': 'cnt_grid_cells_observation_dates'}).head(2)

In [ ]:
# df_cnt_years = df['year'].value_counts(sort=False).to_frame().reset_index()
# df_cnt_years = df_cnt_years.rename(columns={'index': 'year', 'year': 'cnt_grid_cells_dates'})

In [ ]:
# df_cnt_years.head(2)

In [ ]:
# subdir = 'eBird/ebd_output/'

# filename = 'ebd_' + countries_states + '_' + species + '_' + start_date + '_' + end_date + \
# '_complete_zerofilled_grid_cells_proportions_cnt_years_rel' + month + '-' + year_ebird + '.csv'
# print(filename)
        
# df_cnt_years.to_csv(subdir + filename, index=False)

In [ ]:
# species_cnt = 0

# for i in range(len(df_species_codes)):
    
#     print(i)
  
#     species = df_species_codes['species_code'].iloc[i]
#     print(species)
    
#     if ((species == 'souwpw1') | (species == 'bucnig') | (species == 'compoo') | (species == 'whtswi') | 
#         (species == "blkswi") | (species == "treswa")): #
#         continue
    
#     df, df_cnt_checklists, df_cnt_checklists_species, df_prop_greater_than, df_prop_greater_than_cnt, \
# df_subset = get_arrival_days(species, start_date, end_date, month, year_ebird, countries_states)
    
#     species_cnt += 1

In [ ]:
# print(species_cnt)

In [ ]:
# os.system("printf '\a'")